# Classification using Naive Bayes
We load the packages we will use

In [1]:
using CSV
using DataFrames
using MLBase
using DelimitedFiles

Loading the data

In [2]:
DF = readdlm("tennis.csv",',';skipstart = 1);

Now wrangling the data

In [3]:
x1 = DF[:,1]
x2 = DF[:,2]
x3 = DF[:,3]
x4 = DF[:,4]
y = DF[:,5];
#counting unique elements in each column
unique_x1 = unique(x1)
unique_x2 = unique(x2)
unique_x3 = unique(x3)
unique_x4 = unique(x4)

unique_y = unique(y)

#and count the Probability of playing tennis from the Data
len_y = length(y)
len_yes = count(x -> x == "yes", y)
len_no = len_y .-len_yes

prob_yes = len_yes/len_y 
prob_no = len_no/len_y;

#we separate into two matrices related for each yes or no
data_yes = DF[DF[:,5].=="yes",:]
data_no = DF[DF[:,5].=="no",:]

#we count features in data_yes

len_sunny_yes = count(x -> x == unique_x1[1], data_yes)
len_overcast_yes = count(x -> x == unique_x1[2], data_yes)
len_rainy_yes = count(x -> x == unique_x1[3], data_yes)

len_hot_yes = count(x -> x == unique_x2[1], data_yes)
len_mild_yes = count(x -> x == unique_x2[2], data_yes)
len_cool_yes = count(x -> x == unique_x2[3], data_yes)

len_high_yes = count(x -> x == unique_x3[1], data_yes)
len_normal_yes = count(x -> x == unique_x3[2], data_yes)

len_false_yes = count(x -> x == unique_x4[1], data_yes)
len_true_yes = count(x -> x == unique_x4[2], data_yes)

#now the no

len_sunny_no = count(x -> x == unique_x1[1], data_no)
len_overcast_no = count(x -> x == unique_x1[2], data_no)
len_rainy_no = count(x -> x == unique_x1[3], data_yes)

len_hot_no = count(x -> x == unique_x2[1], data_no)
len_mild_no = count(x -> x == unique_x2[2], data_no)
len_cool_no = count(x -> x == unique_x2[3], data_no)

len_high_no = count(x -> x == unique_x3[1], data_no)
len_normal_no = count(x -> x == unique_x3[2], data_no)

len_false_no = count(x -> x == unique_x4[1], data_no)
len_true_no = count(x -> x == unique_x4[2], data_no)


3

Noe we use the Naive Bayes Classifier using:
$$P(yes|NewX)= \frac{P(NewX|yes)P(yes)}{P(NewX)}$$ 

In [4]:
#New predicition 
#p_yes_newX = ["sunny","hot"]

p_yes_newX = 
    (len_sunny_yes / len_yes) *
    (len_hot_yes / len_yes) *
    prob_yes

p_no_newX = 
    (len_sunny_no / len_no) *
    (len_hot_no / len_no) *
    prob_no

#normalize probabilities

@show p_yes_newXN = p_yes_newX / (p_yes_newX + p_no_newX)
@show p_no_newXN = p_no_newX / (p_yes_newX + p_no_newX)

p_yes_newXN = p_yes_newX / (p_yes_newX + p_no_newX) = 0.27027027027027023
p_no_newXN = p_no_newX / (p_yes_newX + p_no_newX) = 0.7297297297297297


0.7297297297297297

New prediciton

In [5]:
#p_yes_new = ["sunny", "cool", "high", "true"]
p_yes_newX = 
    (len_sunny_yes / len_yes) *
    (len_cool_yes / len_yes) *
    (len_high_yes / len_yes) *
    (len_true_yes / len_yes) *
    prob_yes

p_no_newX = 
    (len_sunny_no / len_no) *
    (len_cool_no / len_no) *
    (len_high_no / len_no) *
    (len_true_no / len_no) *
    prob_no
    
@show p_yes_newXN = p_yes_newX / (p_yes_newX + p_no_newX)
@show p_no_newXN = p_no_newX / (p_yes_newX + p_no_newX);

#########################################################################
##################Try to create some function for this###################
#########################################################################

p_yes_newXN = p_yes_newX / (p_yes_newX + p_no_newX) = 0.20458265139116202
p_no_newXN = p_no_newX / (p_yes_newX + p_no_newX) = 0.795417348608838


#### Now with ScikitLearn using PyCall
Load PyCall and the python modules we will need

In [44]:
using PyCall
skl = pyimport("sklearn.naive_bayes")
ModelSelect = pyimport("sklearn.model_selection")

PyObject <module 'sklearn.model_selection' from 'C:\\Users\\PC\\.julia\\conda\\3\\lib\\site-packages\\sklearn\\model_selection\\__init__.py'>

Labeling the strings in the data to train the model

In [45]:
LabelMap=[]
for i=1:5
    push!(LabelMap,labelmap(DF[:,i]))
end
X = zeros(14,4)
for i=1:14
    for j=1:4
        X[i,j]=(labelencode(LabelMap[j],DF[i,j]) .-1);
    end
end
Y = labelencode(LabelMap[5], DF[:,5]) .-1;


Splitting data

In [46]:
X_train, X_test, Y_train, Y_test  = ModelSelect.train_test_split(X,Y);

Selecting model, training, predict and scoring the prediction 

In [53]:
model = skl.GaussianNB()
model.fit(X_train,Y_train)
Prediction = model.predict(X_test)
model.score(X_test,Y_test)


0.75